<a href="https://colab.research.google.com/github/Plananas/AI-and-Data-Mining/blob/main/AI_and_Datamining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link to the data source: https://www.kaggle.com/datasets/carogonzalezgaltier/medical-appointment-scheduling-system

This program will be using the supplied data to reduce the wait times of patients.

In [ ]:
import pandas as pd

In [ ]:
import os
cwd = os.getcwd()
print(cwd)

/content
